In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier


dataset_transfusion = pd.read_table('transfusion.data', sep=',')
print("\nDataset transfusion:")
dataset_transfusion


Dataset transfusion:


,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),Whether he/she donated blood in March 2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0
...,...,...,...,...,...
743,23,2,500,38,0
744,21,2,500,52,0
745,23,3,750,62,0
746,39,1,250,39,0


In [15]:
dataset_transfusion.columns

Index(['Recency (months)', 'Frequency (times)', 'Monetary (c.c. blood)',
       'Time (months)', 'Whether he/she donated blood in March 2007'],
      dtype='object')

In [16]:
transfusion_normalised = dataset_transfusion.values.copy()
normalizador =  StandardScaler()

transfusion_normalised = normalizador.fit_transform(dataset_transfusion)
dataset_transfusion['Recency (months)'] = transfusion_normalised[:,0]
dataset_transfusion['Frequency (times)'] = transfusion_normalised[:,1]
dataset_transfusion['Monetary (c.c. blood)'] = transfusion_normalised[:,2]
dataset_transfusion['Time (months)'] = transfusion_normalised[:,3]

print("\nDataset transfusion Normalized:")
dataset_transfusion


Dataset transfusion Normalized:


,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),Whether he/she donated blood in March 2007
0,-0.927899,7.623346,7.623346,2.615633,1
1,-1.175118,1.282738,1.282738,-0.257881,1
2,-1.051508,1.796842,1.796842,0.029471,1
3,-0.927899,2.482313,2.482313,0.439973,1
4,-1.051508,3.167784,3.167784,1.753579,0
...,...,...,...,...,...
743,1.667904,-0.602307,-0.602307,0.152621,0
744,1.420685,-0.602307,-0.602307,0.727324,0
745,1.667904,-0.430940,-0.430940,1.137826,0
746,3.645659,-0.773675,-0.773675,0.193671,0


In [17]:
transfusion_values = dataset_transfusion.iloc[:,0:4].values
print("\ntransfusion valores dos atributos:\n")
print(transfusion_values)


transfusion valores dos atributos:

[[-0.92789873  7.62334626  7.62334626  2.61563344]
 [-1.17511806  1.28273826  1.28273826 -0.2578809 ]
 [-1.0515084   1.79684161  1.79684161  0.02947053]
 ...
 [ 1.66790417 -0.43093957 -0.43093957  1.13782607]
 [ 3.64565877 -0.77367514 -0.77367514  0.19367135]
 [ 7.72477762 -0.77367514 -0.77367514  1.54832812]]


In [18]:
transfusion_classes = dataset_transfusion.iloc[:,4].values
print("\ntransfusion classes:\n")
print(transfusion_classes)
print("\ntransfusion classes shape:")
print(transfusion_classes.shape)


transfusion classes:

[1 1 1 1 0 0 1 0 1 1 0 0 1 0 1 1 1 1 1 1 1 0 1 1 0 0 0 1 1 0 0 1 1 1 0 1 1
 1 1 1 1 0 1 0 1 1 0 0 0 0 0 1 0 0 1 1 1 1 0 0 0 1 0 1 1 0 1 0 0 0 0 0 1 0
 1 1 1 0 0 0 1 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 1
 0 0 1 0 0 1 0 0 1 1 1 1 1 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0
 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1
 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0
 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 1 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 1 0 0 0 1
 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1
 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0
 0

In [19]:
kfold_treinamento = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=None)
kfold_ajuste_parametros = model_selection.StratifiedKFold(n_splits=3, shuffle=True, random_state=None)

In [20]:
arvore_decisao = DecisionTreeClassifier()
print("====== Iniciando busca pelos melhores parâmetros do algoritmo Arvore de decisao no dataset Transfusion ....\n")
param_dist = {'max_depth':list(np.arange(1,10)), 'min_samples_split':list(np.arange(1,10)) ,'criterion':['entropy'], 'splitter':['best','random']}
grid_search = GridSearchCV(arvore_decisao, param_grid=param_dist, cv=kfold_ajuste_parametros, scoring='accuracy', refit=False)
grid_search.fit(transfusion_values, transfusion_classes)
decisionTreeBestParams = grid_search.best_params_
print("Decision Tree: %s \n\n" % decisionTreeBestParams)

====== Iniciando busca pelos melhores parâmetros do algoritmo Arvore de decisao no dataset Transfusion ....

Decision Tree: {'criterion': 'entropy', 'max_depth': 4, 'min_samples_split': 2, 'splitter': 'best'} 




/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/tree/_classes.py", line 890, in fit
    super().fit(
  File "/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/tree/_classes.py", line 228, in fit
    raise ValueError("min_samples_split must be an integer "
ValueError: min_samples_split must be an integer greater than 1 or a float in (0.0, 1.

In [21]:
knn_vizinhos_proximos = KNeighborsClassifier()
print("====== Iniciando busca pelos melhores parâmetros do algoritmo KNN no dataset Transfusion ....\n")
param_dist = {'n_neighbors':[3,5,7,9], 'metric':['euclidean', 'manhattan'], 'weights':['uniform', 'distance']} #Parâmetros testados
grid_search = GridSearchCV(knn_vizinhos_proximos, param_grid=param_dist, cv=kfold_ajuste_parametros, scoring='accuracy', refit=False)
grid_search.fit(transfusion_values, transfusion_classes)
knnBestParams = grid_search.best_params_
print("KNN: %s \n\n" % knnBestParams)

====== Iniciando busca pelos melhores parâmetros do algoritmo KNN no dataset Transfusion ....

KNN: {'metric': 'euclidean', 'n_neighbors': 9, 'weights': 'uniform'} 




In [22]:
regressao_logistica = LogisticRegression()
print("====== Iniciando busca pelos melhores parâmetros do algoritmo Regressao logistica no dataset Transfusion ....\n")
param_dist = {'penalty':['l2','l1'], 'max_iter':[200,300,400,500],'solver':['liblinear','sag','saga']} #Parâmetros testados
grid_search = GridSearchCV(regressao_logistica, param_grid=param_dist, cv=kfold_ajuste_parametros, scoring='accuracy', refit=False)
grid_search.fit(transfusion_values, transfusion_classes)
regressao_logistica_param = grid_search.best_params_
print("Regressão logistica: %s \n\n" % regressao_logistica_param)


====== Iniciando busca pelos melhores parâmetros do algoritmo Regressao logistica no dataset Transfusion ....

Regressão logistica: {'max_iter': 200, 'penalty': 'l2', 'solver': 'liblinear'} 




/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 442, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties

In [23]:
rede_neural = MLPClassifier()
print("====== Iniciando busca pelos melhores parâmetros do algoritmo Redes neurais no dataset Transfusion ....\n")
param_dist = {'hidden_layer_sizes':list(np.arange(10,20)),'activation':['logistic','identity','tanh'], 'max_iter':[100,200,300,400],'alpha':[0.0001,0.001, 0.01, 0.1], 'solver':['sgd','adam']}
grid_search = GridSearchCV(rede_neural, param_grid=param_dist, cv=kfold_ajuste_parametros, scoring='accuracy', refit=False)
grid_search.fit(transfusion_values, transfusion_classes)
redes_neurais_param = grid_search.best_params_
print("Redes neurais: %s \n\n" % redes_neurais_param)

====== Iniciando busca pelos melhores parâmetros do algoritmo Redes neurais no dataset Transfusion ....

Redes neurais: {'activation': 'tanh', 'alpha': 0.01, 'hidden_layer_sizes': 15, 'max_iter': 300, 'solver': 'adam'} 




/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_pe

In [24]:
predicted_classes = dict()
predicted_classes['Arvore_decisao'] = np.zeros(transfusion_classes.shape)
predicted_classes['KNN_Vizinhos_proximos'] = np.zeros(transfusion_classes.shape)
predicted_classes['Naive_bayes_gaussian'] = np.zeros(transfusion_classes.shape)
predicted_classes['Regressao_logistica'] = np.zeros(transfusion_classes.shape)
predicted_classes['Rede_neural'] = np.zeros(transfusion_classes.shape)

arvore_decisao_best_param = DecisionTreeClassifier(**decisionTreeBestParams)
knn_vizinhos_proximos_best_param = KNeighborsClassifier(**knnBestParams)
naive_bayes_gaussian = GaussianNB()
regressao_logistica_best_param = LogisticRegression(**regressao_logistica_param)
rede_neural_best_param = MLPClassifier(**redes_neurais_param)

In [25]:
for train, test in kfold_treinamento.split(transfusion_values, transfusion_classes):
    data_train, target_train = transfusion_values[train], transfusion_classes[train]
    data_test, target_test = transfusion_values[test], transfusion_classes[test]

    arvore_decisao_best_param = arvore_decisao_best_param.fit(data_train, target_train)
    arvore_decisao_predicted = arvore_decisao_best_param.predict(data_test)
    predicted_classes['Arvore_decisao'][test] = arvore_decisao_predicted

    knn_vizinhos_proximos_best_param = knn_vizinhos_proximos_best_param.fit(data_train, target_train)
    vizinhos_proximos_predicted = knn_vizinhos_proximos_best_param.predict(data_test)
    predicted_classes['KNN_Vizinhos_proximos'][test] = vizinhos_proximos_predicted

    naive_bayes_gaussian = naive_bayes_gaussian.fit(data_train, target_train)
    naive_bayes_gaussian_predicted = naive_bayes_gaussian.predict(data_test)
    predicted_classes['Naive_bayes_gaussian'][test] = naive_bayes_gaussian_predicted

    regressao_logistica_best_param = regressao_logistica_best_param.fit(data_train, target_train)
    regressao_logistica_predicted = regressao_logistica_best_param.predict(data_test)
    predicted_classes['Regressao_logistica'][test] = regressao_logistica_predicted

    rede_neural_best_param = rede_neural_best_param.fit(data_train, target_train)
    rede_neural_predicted = rede_neural_best_param.predict(data_test)
    predicted_classes['Rede_neural'][test] = rede_neural_predicted

In [26]:
for classificador in predicted_classes.keys():
    print("================================================================================================")
    print("Resultados do classificador %s\n%s\n"
          %(classificador, metrics.classification_report(transfusion_classes, predicted_classes[classificador])))
    print("Matriz de confusão: \n%s\n\n\n" % metrics.confusion_matrix(transfusion_classes, predicted_classes[classificador]))


Resultados do classificador Arvore_decisao
              precision    recall  f1-score   support

           0       0.82      0.89      0.85       570
           1       0.51      0.37      0.43       178

    accuracy                           0.77       748
   macro avg       0.66      0.63      0.64       748
weighted avg       0.75      0.77      0.75       748


Matriz de confusão: 
[[508  62]
 [113  65]]



Resultados do classificador KNN_Vizinhos_proximos
              precision    recall  f1-score   support

           0       0.81      0.93      0.87       570
           1       0.59      0.31      0.41       178

    accuracy                           0.78       748
   macro avg       0.70      0.62      0.64       748
weighted avg       0.76      0.78      0.76       748


Matriz de confusão: 
[[532  38]
 [123  55]]



Resultados do classificador Naive_bayes_gaussian
              precision    recall  f1-score   support

           0       0.78      0.92      0.85       570